# Clearwater Modules Architecture

**Author:** Xavier Nogueira

# Installation and Setup

## Install

Carefully follow our **[Installation Instructions](README.md#getting-started)**, especially including:
- Creating a virtual environment for this repository (step 3)

## Import Python Dependancies

In [1]:
import clearwater_modules as cwm
import clearwater_modules.sorter as sorter
import numba
import random
import hvplot.xarray
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Confirm that sub-modules are imported
dir(cwm)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'shared',
 'sorter',
 'tsm',
 'utils']

### If you get `ModuleNotFoundError`:

If you get this error:
```python
ModuleNotFoundError: No module named 'clearwater_modules'
```
Then:
1. Run the following terminal command with your local absolute path to this repo.
    - NOTE: Here we use Jupyter `!` magic command to run from the terminal via this notebook. 
2. Restart the kernel.
3. Rerun the import statements above.

See [4. Add your `ClearWater-modules-python` Path to Miniconda/Anaconda sites-packages](..ReadMe.md#4-add-your-clearwater-modules-python-path-to-minicondaanaconda-sites-packages).

# Writing/using a simple `Model` sub-class example

In this example we will be writing a `base.Model` sub-class that calculates the annual carbon sequestration in a forest for a given year timestep.

**Note:** Do not take the calculation too literally! I got it off ChatGPT in order to find a good, simple example for the code.

## Start by inheriting `base.Model` -> `CarbonSequestration(cwm.base.Model)`

In [2]:
import clearwater_modules as cwm
import clearwater_modules.sorter as sorter
import numba
import random
import hvplot.xarray
import warnings
warnings.filterwarnings("ignore")

initial_state_values = {'water_temp_c': 20, 'surface_area': 1, 'volume': 1}
meteo_parameters = {
    'air_temp_c': 20,
    'q_solar': 400,
    'sed_temp_c': 5,
    'eair_mb': 1,
    'pressure_mb': 1013,
    'cloudiness': .1,
    'wind_speed': 3,
    'wind_a': .3,
    'wind_b': 1.5,
    'wind_c': 1,
    'wind_kh_kw': 1
}

temp_parameters = {
    'use_sed_temp': False,
    'stefan_boltzmann': 0.0000000567,
    'cp_air': 1005,
    'emissivity_water': 0.97,
    'gravity': 9.806,
    'a0': 6984.505294,
    'a1': -188.903931,
    'a2': 2.133357675,
    'a3': -0.01288581,
    'a4': 0.0000439359,
    'a5': -.0000000802392,
    'a6': .0000000000613682,
    'pb': 1600,
    'cps': 1673,
    'h2': 0.1,
    'alphas': 0.0432,
    'richardson_option': True
}

time_step = 1

tsm_model = cwm.tsm.EnergyBudget(
    time_steps=time_step,
    initial_state_values=initial_state_values,  # mandatory
    temp_parameters=temp_parameters,
    meteo_parameters=meteo_parameters,
    track_dynamic_variables=True,  # default is true
    hotstart_dataset=None,  # default is None
    time_dim='tsm_time_step',  # default is "timestep"
)

tsm_model.increment_timestep()
tsm_model.dataset


Initializing from dicts...
Model initialized from input dicts successfully!.


<xarray.Dataset>
Dimensions:            (tsm_time_step: 2, x: 1, y: 1)
Coordinates:
  * tsm_time_step      (tsm_time_step) int32 0 1
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/52)
    water_temp_c       (tsm_time_step, x, y) float64 20.0 20.0
    surface_area       (tsm_time_step, x, y) float64 1.0 1.0
    volume             (tsm_time_step, x, y) float64 1.0 1.0
    use_sed_temp       (x, y) bool True
    stefan_boltzmann   (x, y) float64 5.67e-08
    cp_air             (x, y) int32 1005
    ...                 ...
    q_sediment         (tsm_time_step, x, y) float64 nan -401.5
    dTdt_sediment_c    (tsm_time_step, x, y) float64 nan 129.6
    q_net              (tsm_time_step, x, y) float64 nan -151.1
    q_longwave_down    (tsm_time_step, x, y) float64 nan 337.8
    q_longwave_up      (tsm_time_step, x, y) float64 nan 406.2
    dTdt_water_c       (tsm_time_step, x, y) float64 nan -3.619e-05

In [3]:
tsm_model.increment_timestep()
tsm_model.dataset

KeyError: "not all values found in index 'tsm_time_step'. Try setting the `method` keyword argument (example: method='nearest')."

In [2]:
tsm_model.get_state_variables()

[Variable(name='water_temp_c', long_name='Water temperature', units='degC', description='TSM state variable for water temperature', use='state', process=CPUDispatcher(<function t_water_c at 0x0000017B72896700>)),
 Variable(name='surface_area', long_name='Surface area', units='m^2', description='Surface area', use='state', process=<function mock_surface_area at 0x0000017B6F68CD60>),
 Variable(name='volume', long_name='Volume', units='m^3', description='Volume', use='state', process=<function mock_volume at 0x0000017B72896C00>)]

In [3]:
print('Variable | Inputs\n------------------')
for i in tsm_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

Variable | Inputs
------------------
air_temp_k | ['air_temp_c']
water_temp_k | ['water_temp_c']
mixing_ratio_air | ['eair_mb', 'pressure_mb']
density_air | ['pressure_mb', 'air_temp_k', 'mixing_ratio_air']
density_water | ['water_temp_c']
esat_mb | ['water_temp_k', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
density_air_sat | ['water_temp_k', 'esat_mb', 'pressure_mb']
ri_number | ['gravity', 'density_air', 'density_air_sat', 'wind_speed']
ri_function | ['ri_number']
lv | ['water_temp_k']
cp_water | ['water_temp_c']
emissivity_air | ['air_temp_k']
wind_function | ['ri_function', 'wind_a', 'wind_b', 'wind_c', 'wind_speed']
q_latent | ['pressure_mb', 'density_water', 'lv', 'wind_function', 'esat_mb', 'eair_mb']
q_sensible | ['wind_kh_kw', 'cp_air', 'density_water', 'wind_function', 'air_temp_k', 'water_temp_k']
q_sediment | ['use_sed_temp', 'pb', 'cps', 'alphas', 'h2', 'sed_temp_c', 'water_temp_c']
dTdt_sediment_c | ['use_sed_temp', 'alphas', 'h2', 'water_temp_c', 'sed_temp_c']
q_longwave_

In [4]:
tsm_model.increment_timestep()
tsm_model.dataset

<xarray.Dataset>
Dimensions:            (year: 2, x: 1, y: 1)
Coordinates:
  * year               (year) int32 0 1
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/52)
    water_temp_c       (year, x, y) float64 20.0 -7.427e+10
    surface_area       (year, x, y) int32 1 1
    volume             (year, x, y) int32 1 1
    use_sed_temp       (x, y) bool True
    stefan_boltzmann   (x, y) int32 10
    cp_air             (x, y) int32 1
    ...                 ...
    q_sediment         (year, x, y) float64 nan -2.572e+03
    dTdt_sediment_c    (year, x, y) float64 nan 0.006001
    q_longwave_down    (year, x, y) float64 nan 5.958e+10
    q_longwave_up      (year, x, y) float64 nan 7.386e+10
    q_net              (year, x, y) float64 nan -3.1e+17
    dTdt_water_c       (year, x, y) float64 nan -7.427e+10

## Next, use the `register_variable` decorator to add a few variables

To do this, make a sub-class of `base.Variable` but with the decorator pointed at the model(s) you want to add the variables too. Note that the `models` argument of the decorator must be either a single sub-class of `base.Model`, or a list of them.

Next, just write instances of the new `base.Variable` sub-class. Each variable's `use` attribute must be set to `static`, `dynamic`, and `state`. Read below about what this means / how you should split up your variables.

Note that anything that needs to be calculated or input into a model should be encapsulated by a variable!

In [4]:
@cwm.base.register_variable(models=CarbonSequestration)
class Variable(cwm.base.Variable):
    ...

In [5]:
Variable(
    name='npp',
    long_name='Net Primary Productivity (NPP)',
    units='g/m^2/year',
    description='The annual average NPP of the forest ecosystem.',
    use='static',
)
Variable(
    name='carbon_content',
    long_name='Carbon Content ratio',
    units='ratio',
    description='The fraction of NPP that is composed of carbon (usually around 50%, but it can vary).',
    use='static',
)

# display the variables we have registered so far
display(CarbonSequestration.get_variable_names())

['npp', 'carbon_content']

### Add our static variables

**Working Definition:** Static variables are any variables that will not change across the course of a simulation, regardless of how many time-steps are run.

Note that one can update static variables if they really want to by re-initializing the model class and providing new static variable inputs.

Here we will use the following static variables:
1. **Net Primary Productivity (NPP)**: The average annual NPP of the forest ecosystem (g/m²/year).
2. **Carbon Content**: The fraction of NPP that is composed of carbon (usually around 50%, but it can vary).

### Add our dynamic variables

**Working Definition:** Dynamic variables are any intermediate variable calculation that don't need to be passed to the next timestep. All dynamic variables need to be associated with a function via the optional `Variable.process` attribute. This "process" function is used to calculate them. **Importantly, the arguments of said function should match the variable names that will be passed in!**

In this simple example we will have only one dynamic variable:
1. **Annual carbon sequestration** (delta_C_annual):

   `delta_C_annual = npp * carbon_content * forest_area`

In [6]:
@numba.njit
def delta_C_annual(
    npp: float,
    carbon_content: float,
    forest_area: float,
) -> float:
    return npp * carbon_content * forest_area

In [7]:
Variable(
    name='delta_C_annual',
    long_name='Annual Carbon Delta',
    units='g',
    description='Annual change in forest carbon content',
    use='dynamic',
    process=delta_C_annual,
)

# display the variables we have registered so far
display(CarbonSequestration.get_variable_names())

['npp', 'carbon_content', 'delta_C_annual']

### Add our state variable

**Working Definition:** A state variable is the main input/output to each timestep. Notably, it can be updated between timesteps to allow interaction with other models. Our model needs to be initialized with state variable values, and no matter what settings are used in initialization, the state variable is stored in our main dataset (keep reading to see this).

Our state variable is the total carbon stock of the forest, which is updated each year:
1. **Total carbon stock** (C_total):

    `C_total = C_total + delta_C_annual`
    
State variables also require a process function.

In [8]:
@numba.njit
def C_total(
    C_total: float,
    delta_C_annual: float,
) -> float:
    return C_total + delta_C_annual

def forest_area(
    forest_area: float,
) -> float:
    return forest_area

In [9]:
Variable(
    name='C_total',
    long_name='Carbon total',
    units='g',
    description='Total forest carbon content',
    use='state',
    process=C_total,
)

Variable(
    name='forest_area',
    long_name='Area of the forest',
    units='m^2',
    description='Area of the forest, may change year by year with deforestation.',
    use='state',
    process=forest_area,
)

# display the variables we have registered so far
display(CarbonSequestration.get_variable_names())

['npp', 'carbon_content', 'delta_C_annual', 'C_total', 'forest_area']

In [10]:
# for state variables we can see them before initialization
display(CarbonSequestration.get_state_variables())

[Variable(name='C_total', long_name='Carbon total', units='g', description='Total forest carbon content', use='state', process=CPUDispatcher(<function C_total at 0x0000023BF7C7A340>)),
 Variable(name='forest_area', long_name='Area of the forest', units='m^2', description='Area of the forest, may change year by year with deforestation.', use='state', process=<function forest_area at 0x0000023BFDB05800>)]

## Now let's instantiate our new model

To instantiate a model we need to pass in a dictionary with our initial state variable values, any non-default changes to our static variables, and any other optional config settings.

In [11]:
initial_state_values = {'C_total': 1000, 'forest_area': 1000}
static_variable_values = {
    'carbon_content': 0.5,
    'npp': 10,
}

carbon_model = CarbonSequestration(
    time_steps = 100, # mandatory
    initial_state_values=initial_state_values,  # mandatory
    static_variable_values=static_variable_values,  # mandatory/optional depending on defaults
    track_dynamic_variables=True,  # default is false
    hotstart_dataset=None,  # default is None
    time_dim='year',  # default is "timestep"
)

Initializing from dicts...
Model initialized from input dicts successfully!.


### All instantiated models have static, dynamic, and state variable properties

In [12]:
display(carbon_model.state_variables)

[Variable(name='C_total', long_name='Carbon total', units='g', description='Total forest carbon content', use='state', process=CPUDispatcher(<function C_total at 0x0000023BF7C7A340>)),
 Variable(name='forest_area', long_name='Area of the forest', units='m^2', description='Area of the forest, may change year by year with deforestation.', use='state', process=<function forest_area at 0x0000023BFDB05800>)]

In [13]:
display(carbon_model.static_variables)

[Variable(name='npp', long_name='Net Primary Productivity (NPP)', units='g/m^2/year', description='The annual average NPP of the forest ecosystem.', use='static', process=None),
 Variable(name='carbon_content', long_name='Carbon Content ratio', units='ratio', description='The fraction of NPP that is composed of carbon (usually around 50%, but it can vary).', use='static', process=None)]

In [14]:
display(carbon_model.dynamic_variables)

[Variable(name='delta_C_annual', long_name='Annual Carbon Delta', units='g', description='Annual change in forest carbon content', use='dynamic', process=CPUDispatcher(<function delta_C_annual at 0x0000023BFDB063E0>))]

### One can access their "computation order" which is calculated using a "dependency tree" approach in `sorter.py`

In [15]:
carbon_model.computation_order

[Variable(name='delta_C_annual', long_name='Annual Carbon Delta', units='g', description='Annual change in forest carbon content', use='dynamic', process=CPUDispatcher(<function delta_C_annual at 0x0000023BFDB063E0>)),
 Variable(name='C_total', long_name='Carbon total', units='g', description='Total forest carbon content', use='state', process=CPUDispatcher(<function C_total at 0x0000023BF7C7A340>)),
 Variable(name='forest_area', long_name='Area of the forest', units='m^2', description='Area of the forest, may change year by year with deforestation.', use='state', process=<function forest_area at 0x0000023BFDB05800>)]

In [16]:
print('Variable | Inputs\n------------------')
for i in carbon_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

Variable | Inputs
------------------
delta_C_annual | ['npp', 'carbon_content', 'forest_area']
C_total | ['C_total', 'delta_C_annual']
forest_area | ['forest_area']


### Data is stored in `self.dataset`

In [17]:
carbon_model.dataset

<xarray.Dataset>
Dimensions:         (year: 101, x: 1, y: 1)
Coordinates:
  * year            (year) int32 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * x               (x) float64 1.0
  * y               (y) float64 1.0
Data variables:
    C_total         (year, x, y) float64 1e+03 nan nan nan ... nan nan nan nan
    forest_area     (year, x, y) float64 1e+03 nan nan nan ... nan nan nan nan
    npp             (x, y) int32 10
    carbon_content  (x, y) float64 0.5
    delta_C_annual  (year, x, y) float64 nan nan nan nan nan ... nan nan nan nan

We can see that the dataset is initialized with a time dimension for any state variables, from years 0 (initial conditions) to 100 (final timestep). State variables (`C_total` and `forest_area`) have a value filled in for initial conditions based on what we provided during the model instantiation step. Static variables (`npp`, `carbon_content`) do not have a time dimension since they do not vary temporally. In this case, since we opt to track dynamic variables, we can see that our dynamic variable (`delta_C_annual`) also has a time dimension. If we had set `track_dynamic_variables` to `False` during model instantiation, this variable would be excluded from our model output. 

## Running a timestep
All timesteps can be run independently. Optionally, one can update the state values with a float or a `xarray.DataArray`.

In [18]:
carbon_model.increment_timestep()
carbon_model.dataset

<xarray.Dataset>
Dimensions:         (year: 101, x: 1, y: 1)
Coordinates:
  * year            (year) int32 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * x               (x) float64 1.0
  * y               (y) float64 1.0
Data variables:
    C_total         (year, x, y) float64 1e+03 6e+03 nan nan ... nan nan nan nan
    forest_area     (year, x, y) float64 1e+03 1e+03 nan nan ... nan nan nan nan
    npp             (x, y) int32 10
    carbon_content  (x, y) float64 0.5
    delta_C_annual  (year, x, y) float64 nan 5e+03 nan nan ... nan nan nan nan

Now we can see that the second timestep value has been filled in for our state and dynamic variables in the dataset above.

## Running a loop of timesteps

Here we run 100 years of our model with the following hypothetical:
* For the first 50 years deforestation reduces forest area incrementally.
* 50 years in, a program begins that ends deforestation, and the forest grows back incrementally.

**This demonstrates how we can update state variables to interact with other models!**

In [19]:
%%time
for i in range(2,101):
    forest_area_change = random.uniform(0.0, 25)
    if i < 50:
        forest_area_change = -forest_area_change
    new_forest_area = (carbon_model.dataset.forest_area + forest_area_change).isel(year=i-1)
    carbon_model.increment_timestep(update_state_values={'forest_area': new_forest_area})
carbon_model.dataset

CPU times: total: 1.06 s
Wall time: 1.13 s


<xarray.Dataset>
Dimensions:         (year: 101, x: 1, y: 1)
Coordinates:
  * year            (year) int32 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * x               (x) float64 1.0
  * y               (y) float64 1.0
Data variables:
    C_total         (year, x, y) float64 1e+03 6e+03 ... 3.197e+05 3.241e+05
    forest_area     (year, x, y) float64 1e+03 1e+03 989.5 ... 865.0 866.3 867.4
    npp             (x, y) int32 10
    carbon_content  (x, y) float64 0.5
    delta_C_annual  (year, x, y) float64 nan 5e+03 ... 4.331e+03 4.337e+03

In [20]:
carbon_model.dataset.hvplot(x='year', y='delta_C_annual', title='delta_C_annual')

:DynamicMap   [y,x]
   :Curve   [year]   (delta_C_annual)

In [21]:
carbon_model.dataset.hvplot(x='year', y='C_total', title='C_total')

:DynamicMap   [y,x]
   :Curve   [year]   (C_total)

# TSM `EnergyBudget` Example

Now that we understand how the code architecture works, we can explore a real example.

In [22]:
from clearwater_modules.tsm.model import EnergyBudget

In [23]:
# Confirm that sub-modules are imported
dir(cwm.tsm)

['EnergyBudget',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'constants',
 'dynamic_variables',
 'model',
 'processes',
 'state_variables',
 'static_variables']

## Start by instantiating a `EnergyBudget`

Initial state variable values are always required. To see the names/info of a model's state variables, we can use `Model.get_state_variables()`.

In [24]:
EnergyBudget.get_state_variables()

[Variable(name='water_temp_c', long_name='Water temperature', units='degC', description='TSM state variable for water temperature', use='state', process=CPUDispatcher(<function t_water_c at 0x0000023BF8322980>)),
 Variable(name='surface_area', long_name='Surface area', units='m^2', description='Surface area', use='state', process=<function mock_surface_area at 0x0000023BF51B6E80>),
 Variable(name='volume', long_name='Volume', units='m^3', description='Volume', use='state', process=<function mock_volume at 0x0000023BF8322E80>)]

In [47]:
initial_state_values = {
    'water_temp_c': 1.0,
    'volume': 1.0,
    'surface_area': 1.0,
}

time_steps = 5

In [48]:
my_model = EnergyBudget(
    time_steps,
    initial_state_values,
    time_dim='my_time_step'
)

Initializing from dicts...
Model initialized from input dicts successfully!.


In [49]:
[i for i in dir(my_model) if i[0] != '_']

['all_variables',
 'computation_order',
 'dataset',
 'dynamic_variables',
 'dynamic_variables_names',
 'get_state_variables',
 'get_variable',
 'get_variable_names',
 'hotstart_dataset',
 'increment_timestep',
 'initial_state_values',
 'met_parameters',
 'register_variable',
 'state_variables',
 'state_variables_names',
 'static_variable_values',
 'static_variables',
 'static_variables_names',
 'temp_parameters',
 'temporal_variables',
 'time_dim',
 'time_steps',
 'timestep',
 'track_dynamic_variables',
 'unregister_variables',
 'updateable_static_variables']

## TSM can be initialized with alternative met/temp parameter
**This is an example of a model specific `__init__`**. As of now we are using the defaults.

In [50]:
my_model.met_parameters

{'air_temp_c': 20.0,
 'q_solar': 400.0,
 'sed_temp_c': 5.0,
 'eair_mb': 1.0,
 'pressure_mb': 1013.0,
 'cloudiness': 0.1,
 'wind_speed': 3.0,
 'wind_a': 0.3,
 'wind_b': 1.5,
 'wind_c': 3.0,
 'wind_kh_kw': 1.0}

In [51]:
my_model.temp_parameters

{'stefan_boltzmann': 5.67e-08,
 'cp_air': 1005.0,
 'emissivity_water': 0.97,
 'gravity': -9.806,
 'a0': 6984.505294,
 'a1': -188.903931,
 'a2': 2.133357675,
 'a3': -0.01288580973,
 'a4': 4.393587233e-05,
 'a5': -8.023923082e-08,
 'a6': 6.136820929e-11,
 'pb': 1600.0,
 'cps': 1673.0,
 'h2': 0.1,
 'alphas': 0.0432,
 'richardson_option': True}

In [52]:
my_model.time_dim

'my_time_step'

## All models have static, dynamic, and state variables

In [53]:
display(my_model.static_variables)

[Variable(name='use_sed_temp', long_name='Use Sediment Temperature?', units='boolean', description='Controls whether to use/calculate sediment temperature or not.', use='static', process=None),
 Variable(name='stefan_boltzmann', long_name='Stefan-Boltzmann Constant', units='W m-2 K-4', description='The Stefan-Boltzmann constant.', use='static', process=None),
 Variable(name='cp_air', long_name='Specific Heat Capacity of Air', units='J kg-1 K-1', description='The specific heat capacity of air.', use='static', process=None),
 Variable(name='emissivity_water', long_name='Emissivity of Water', units='1', description='The emissivity of water.', use='static', process=None),
 Variable(name='gravity', long_name='Gravity', units='m s-2', description='The acceleration due to gravity.', use='static', process=None),
 Variable(name='a0', long_name='Albedo of Water', units='unitless', description='The albedo of water.', use='static', process=None),
 Variable(name='a1', long_name='Albedo of Water', u

In [54]:
display(my_model.dynamic_variables)

[Variable(name='air_temp_k', long_name='Air temperature', units='K', description='Air temperature', use='dynamic', process=CPUDispatcher(<function air_temp_k at 0x0000023BF8303240>)),
 Variable(name='water_temp_k', long_name='Water temperature', units='K', description='Water temperature', use='dynamic', process=CPUDispatcher(<function water_temp_k at 0x0000023BF83032E0>)),
 Variable(name='mixing_ratio_air', long_name='Mixing ratio of air', units='unitless', description='Mixing ratio of air', use='dynamic', process=CPUDispatcher(<function mixing_ratio_air at 0x0000023BF8303420>)),
 Variable(name='density_air', long_name='Density of air', units='kg/m^3', description='Density of air', use='dynamic', process=CPUDispatcher(<function density_air at 0x0000023BF83025C0>)),
 Variable(name='density_water', long_name='Density of water', units='kg/m^3', description='Density of water', use='dynamic', process=CPUDispatcher(<function mf_density_water at 0x0000023BF8321DA0>)),
 Variable(name='esat_mb'

## One can access their "computation order" which is calculated using a "dependency tree" approach in `sorter.py`

In [55]:
my_model.computation_order

[Variable(name='air_temp_k', long_name='Air temperature', units='K', description='Air temperature', use='dynamic', process=CPUDispatcher(<function air_temp_k at 0x0000023BF8303240>)),
 Variable(name='water_temp_k', long_name='Water temperature', units='K', description='Water temperature', use='dynamic', process=CPUDispatcher(<function water_temp_k at 0x0000023BF83032E0>)),
 Variable(name='mixing_ratio_air', long_name='Mixing ratio of air', units='unitless', description='Mixing ratio of air', use='dynamic', process=CPUDispatcher(<function mixing_ratio_air at 0x0000023BF8303420>)),
 Variable(name='density_air', long_name='Density of air', units='kg/m^3', description='Density of air', use='dynamic', process=CPUDispatcher(<function density_air at 0x0000023BF83025C0>)),
 Variable(name='density_water', long_name='Density of water', units='kg/m^3', description='Density of water', use='dynamic', process=CPUDispatcher(<function mf_density_water at 0x0000023BF8321DA0>)),
 Variable(name='esat_mb'

In [56]:
for i in my_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

air_temp_k | ['air_temp_c']
water_temp_k | ['water_temp_c']
mixing_ratio_air | ['eair_mb', 'pressure_mb']
density_air | ['pressure_mb', 'air_temp_k', 'mixing_ratio_air']
density_water | ['water_temp_c']
esat_mb | ['water_temp_k', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
density_air_sat | ['water_temp_k', 'esat_mb', 'pressure_mb']
ri_number | ['gravity', 'density_air', 'density_air_sat', 'wind_speed']
ri_function | ['ri_number']
lv | ['water_temp_k']
cp_water | ['water_temp_c']
emissivity_air | ['air_temp_k']
wind_function | ['ri_function', 'wind_a', 'wind_b', 'wind_c', 'wind_speed']
q_latent | ['pressure_mb', 'density_water', 'lv', 'wind_function', 'esat_mb', 'eair_mb']
q_sensible | ['wind_kh_kw', 'cp_air', 'density_water', 'wind_function', 'air_temp_k', 'water_temp_k']
q_sediment | ['use_sed_temp', 'pb', 'cps', 'alphas', 'h2', 'sed_temp_c', 'water_temp_c']
dTdt_sediment_c | ['use_sed_temp', 'alphas', 'h2', 'water_temp_c', 'sed_temp_c']
q_longwave_down | ['air_temp_k', 'emissivity_air

## Run 5 timesteps

In [57]:
@numba.jit(forceobj=True)
def run_n_timesteps(time_steps: int, model: EnergyBudget):
    for i in range(time_steps):
        model.increment_timestep()

In [58]:
%%time
run_n_timesteps(time_steps, my_model)

CPU times: total: 6.47 s
Wall time: 7.12 s


In [59]:
my_model.dataset

<xarray.Dataset>
Dimensions:            (my_time_step: 6, x: 1, y: 1)
Coordinates:
  * my_time_step       (my_time_step) int32 0 1 2 3 4 5
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/52)
    water_temp_c       (my_time_step, x, y) float64 1.0 1.0 1.0 1.0 1.001 1.001
    volume             (my_time_step, x, y) float64 1.0 1.0 1.0 1.0 1.0 1.0
    surface_area       (my_time_step, x, y) float64 1.0 1.0 1.0 1.0 1.0 1.0
    use_sed_temp       (x, y) bool True
    stefan_boltzmann   (x, y) float64 5.67e-08
    cp_air             (x, y) float64 1.005e+03
    ...                 ...
    q_sediment         (my_time_step, x, y) float64 nan 107.1 ... 107.1 107.1
    dTdt_sediment_c    (my_time_step, x, y) float64 nan -34.56 ... -34.56 -34.55
    q_net              (my_time_step, x, y) float64 nan 659.1 ... 659.1 659.1
    q_longwave_down    (my_time_step, x, y) float64 nan 337.8 ... 337.8 337.8
    q_longwave_up      (my_time_step, x, y) float64 nan 310.7 ... 310.7 310.7
    dTdt_water_c       (my_time_step, x, y) float64 nan 0.0001569 ... 0.0001569